In [2]:
import pandas as pd
import openai

In [10]:
df = pd.read_csv('../data/train.csv')

In [11]:
df.drop(columns=["input", "text"], axis=1, inplace=True)
df.rename(columns={"instruction":"user", "output":"assistant"}, inplace=True)

In [12]:
df.to_csv("../data/openai_data.csv", index=False)

In [15]:
import csv
import json

csv_file_path = '../data/openai_data.csv'
jsonl_file_path = '../data/openai_data1.jsonl'

with open(csv_file_path, mode='r', encoding='utf-8') as csv_file, \
     open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        json_object = json.dumps({"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."},
                                               {"role": "user", "content": row["user"]},
                                               {"role": "assistant", "content": row["assistant"]}]})
        # json_object = json.dumps({"prompt": row["prompt"], "completion": row["completion"]})
        jsonl_file.write(json_object + '\n')

print("변환 완료: CSV -> JSONL")

변환 완료: CSV -> JSONL


In [3]:
openai.File.list()

APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [16]:
from dotenv import dotenv_values
from langchain.llms import HuggingFacePipeline
import os

env_vars = dotenv_values('../../.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = env_vars.get('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = env_vars.get('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = env_vars.get('LANGCHAIN_API_KEY')

In [17]:
from openai import OpenAI

client = OpenAI()

client.files.create(
  file=open("../data/openai_data1.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-LLsRnUtl7c51nkW1EYRJsUpz', bytes=7228215, created_at=1705556376, filename='openai_data1.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [23]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file="../data/openai_data.jsonl", 
  model="gpt-3.5-turbo", 
  hyperparameters={
    "n_epochs":2
  }
)

BadRequestError: Error code: 400 - {'error': {'message': 'invalid training_file: ../data/openai_data.jsonl', 'type': 'invalid_request_error', 'param': 'training_file', 'code': None}}

In [20]:
import openai
import wandb

# WandB 설정
wandb.init(project="openai-finetuning", entity="crysis")


# 파인튜닝에 사용할 데이터셋의 경로
train_file = "file-LLsRnUtl7c51nkW1EYRJsUpz"

# 파인튜닝 설정
finetune_config = {
    "training_file": "../data/openai_data1.jsonl",
    "model": "gpt-3.5-turbo"
}

# 파인튜닝 실행
print("파인튜닝 시작...")
finetune_result = openai.fine_tuning.jobs.create(**finetune_config)

# WandB에 파인튜닝 정보 기록
wandb.log(finetune_config)
wandb.log({"finetune_result": finetune_result})

print("파인튜닝 완료.")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


model,gpt-3.5-turbo
training_file,file-LLsRnUtl7c51nkW...


파인튜닝 시작...


BadRequestError: Error code: 400 - {'error': {'message': 'invalid training_file: ../data/openai_data1.jsonl', 'type': 'invalid_request_error', 'param': 'training_file', 'code': None}}